In [5]:
import pandas as pd

# Schlagwort-Datei laden (mit Spalten: Schlagwort, Gewicht, Systematik, Alleinstehend)
schlagworte_df = pd.read_excel("schlagworte_systematik_gewichtung.xlsx")
schlagworte_df.columns = schlagworte_df.columns.str.strip()



# Dictionaries für schnelleren Zugriff
weights = dict(zip(schlagworte_df["Schlagwort"], schlagworte_df["Gewichtung"]))
systematics = dict(zip(schlagworte_df["Schlagwort"], schlagworte_df["Systematik"]))


# O-Sätze laden
saetze_df = pd.read_excel("o-sätze.xlsx")
saetze_df.columns = saetze_df.columns.str.strip()

# Alle Spalten außer "IDN" durchsuchen
spalten_zum_pruefen = [col for col in saetze_df.columns if col != "IDN"]

def evaluate_row(row):
    text_all = " ".join(str(val) for val in row if not pd.isna(val)).lower()
    word_counts = {}

    for word in weights.keys():
        count = text_all.count(word.lower())
        if count > 0:
            gewicht = weights.get(word, 1)
            score = count * gewicht
            word_counts[word] = (count, score, systematics.get(word, ""))

    if not word_counts:
        return 0, "", 0, ""

    total_weight = sum(weights.get(w, 1) for w in word_counts.keys())

    # Entscheidungskriterien
    if len(word_counts) >= 3 or total_weight >= 3:
        formatted = [
            f"{word} ({count} Treffer, Gewicht={weights.get(word,1)}, Score={score}, Systematik={syst})"
            for word, (count, score, syst) in word_counts.items()
        ]
        return len(word_counts), "; ".join(formatted), total_weight, "OK"



# Neue Spalten erzeugen
saetze_df[["Count", "Gefundene_Schlagworte", "Gesamtgewichtung", "Status"]] = saetze_df[spalten_zum_pruefen].apply(
    lambda row: pd.Series(evaluate_row(row)), axis=1
)

# Nur Zeilen mit Status=OK übernehmen
ergebnis_df = saetze_df[saetze_df["Status"] == "OK"][["IDN", "Titel", "Count", "Gesamtgewichtung", "Gefundene_Schlagworte"]]

# Ergebnis speichern
ergebnis_df.to_excel("gefilterte_saetze.xlsx", index=False)

print("Fertig! Datei 'gefilterte_saetze.xlsx' erstellt ✅")


Fertig! Datei 'gefilterte_saetze.xlsx' erstellt ✅
